In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"}
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"}
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"}
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"}
    )
]


In [2]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm = ChatGroq(groq_api_key=groq_api_key,model="llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022D792BC7A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022D793DBA40>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

ModuleNotFoundError: No module named 'langchain_huggingface'

In [ ]:
## VectorStores
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents,embedding=embeddings)
vectorstore

In [ ]:
vectorstore.similarity_search("cat")

[Document(id='791c7b51-15d5-4c78-b869-bbef5bb565ff', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='a382a0b6-7185-4dcd-953c-374b8d1bd954', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='d6d8a7cf-f181-4e15-9153-a020e6e63594', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(id='28f648fa-1974-4692-bd24-29daa9c7ef12', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

In [ ]:
## Async query
await vectorstore.asimilarity_search("cat")

[Document(id='791c7b51-15d5-4c78-b869-bbef5bb565ff', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='a382a0b6-7185-4dcd-953c-374b8d1bd954', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='d6d8a7cf-f181-4e15-9153-a020e6e63594', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(id='28f648fa-1974-4692-bd24-29daa9c7ef12', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

In [ ]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='791c7b51-15d5-4c78-b869-bbef5bb565ff', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351057410240173),
 (Document(id='a382a0b6-7185-4dcd-953c-374b8d1bd954', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.574089765548706),
 (Document(id='d6d8a7cf-f181-4e15-9153-a020e6e63594', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.665792465209961),
 (Document(id='28f648fa-1974-4692-bd24-29daa9c7ef12', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  1.8011279106140137)]

### Retrievers

In [ ]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='791c7b51-15d5-4c78-b869-bbef5bb565ff', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='a382a0b6-7185-4dcd-953c-374b8d1bd954', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [ ]:
retriever=vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":1}
)
retriever.batch(["cat","dog"])

[[Document(id='791c7b51-15d5-4c78-b869-bbef5bb565ff', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='a382a0b6-7185-4dcd-953c-374b8d1bd954', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [ ]:
## RAG
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """ 
Answer this question using the provided context only . 

{question}

context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human",message)])

rag_chain = {"context": retriever,"question":RunnablePassthrough()}|prompt|llm
response = rag_chain.invoke("tell me about dog")
print(response.content)

According to the provided context, dogs are great companions, known for their loyalty and friendliness.
